In [1]:
import numpy as np   # Importo paquete de álgebra lineal
import matplotlib.pyplot as plt   # Importo paquete de gráficos
import copy
import mef
plt.rc('figure', figsize=(15,8))   # Para gráficos
plt.rc('font',size=22)             # Para gráficos

# Punto 5

Determine los desplazamientos de los nodos y sus pendientes; las fuerzas en cada elemento y las reacciones. Tome $E = 29 x 10^{6}\: psi$ (libras por pulgada cuadrada) e $I = 200\: in^4$.
<center><img src="ImgProb05.png"></center>

In [2]:
# Defino los grados de libertad por nodo "glxn".
glxn = 2

In [3]:
# Defino el módulo de elasticidad "E" de la viga.
E = 29e6   # psi=lb/in^2
# Transformo 1 ft = 12 in.
E = E*(12**2)   # lb/ft^2

# Defino el momento de inercia "I" de la viga.
I = 200   # in^4
# Transformo 1 ft = 12 in.
I = I/(12**4)   # ft^4

# Defino la longitud "L" de la viga.
L = 15   # ft

# Defino el valor de la carga distribuida "w".
w = 200   # lb/ft

In [4]:
# Defino la matriz de nodos "MN".
# - Columna 1 es la coordenada "x" del nodo.
# - Columna 2 es la coordenada "y" del nodo.
# - Columna 3 es la coordenada "z" del nodo.
MN = np.array([[0,0,0],
               [15,0,0],
               [30,0,0]])

# Su número de filas es el número de NODOS "Nn".
Nn = MN.shape[0]

In [5]:
# Defino la matriz de conectividad "MC".
# Informa qué nodos componen a cada elemento.
MC= np.array([[0,1],
              [1,2]])

# Su número de filas es el número de ELEMENTOS "Ne".
# Su número de columnas es el número NODOS POR ELEMENTO "Nnxe".
Ne, Nnxe = MC.shape

In [6]:
# Defino la longitud "Le" de cada elemento.
Le = L/Ne

## Viga, $Ne = 1$ elemento, $Nn = 2$ nodos, con carga distribuida $w$

La carga distribuida $w$ se traslada a los nodos $0$ y $1$ de la siguiente manera:
$$
f_0 = \frac{-w{L_e}}{2} \hspace{0.5cm} ; \hspace{0.5cm} m_0 = \frac{-w{L_e}^2}{12} \hspace{0.5cm} ; \hspace{0.5cm} f_1 = \frac{-w{L_e}}{2} \hspace{0.5cm} ; \hspace{0.5cm} m_1 = \frac{w{L_e}^2}{12}
$$
<center><img src="viga.png"></center>

## Viga, $Ne = 2$ elementos, $Nn = 3$ nodos, con carga distribuida $w$

La carga distribuida $w$ se traslada a los nodos $0$, $1$ y $2$ de la siguiente manera:
$$
f_0 = \frac{-w{L_e}}{2} \hspace{0.5cm} ; \hspace{0.5cm} m_0 = \frac{-w{L_e}^2}{12} \hspace{0.5cm} ; \hspace{0.5cm} f_1 = -w{L_e} \hspace{0.5cm} ; \hspace{0.5cm} m_1 = 0 \hspace{0.5cm} ; \hspace{0.5cm} f_2 = \frac{-w{L_e}}{2} \hspace{0.5cm} ; \hspace{0.5cm} m_2 = \frac{w{L_e}^2}{12}
$$

In [7]:
# Defino vector "fm" que almacena fuerzas y momentos trasladados a los nodos, en índices pares e impares respectivamente.
fm = np.zeros([2*(Ne+1)])

fm[0] = -w*Le/2
fm[1] = -(w*Le**2)/12
fm[2] = -w*Le
fm[3] = 0
fm[4] = -w*Le/2
fm[5] = (w*Le**2)/12

### Condiciones de vínculo en DESPLAZAMIENTO y ROTACIÓN
- $d_0 = 0\: m$ (empotramiento)
- $\phi_0 = 0$
- $d_2 = 0\: m$

### Condiciones de vínculo en FUERZA y TORQUE
- $F_1 = f_1$ (debida a la carga distribuida)
- $M_1 = m_1$ (debida a la carga distribuida)
- $M_2 = m_2$ (debida a la carga distribuida)

In [8]:
# Defino vector "s" que contiene los nodos con condiciones de vínculo en desplazamiento y ángulo.
s = np.array([0,1,4])

# Defino vector "Us" con los valores de las condiciones de vínculo.
Us = [[0],[0],[0]]

# Defino vector "r" que contiene los nodos con condiciones de vínculo en fuerza y torque. Es el complemento de "s".
r = np.array([i for i in range(Nn*glxn) if i not in s])

# Defino vector "Fr" con los valores de las condiciones de vínculo.
Fr = np.array([[fm[i]] for i in r])

## Resolución

In [9]:
archivo= 'Matrices_elementales.txt'
with open(archivo,'w') as f:   # Creo archivo desde cero, por eso uso "w".
    f.write('Matrices Elementales\n ===============')
archivo1= 'Matriz_global.txt'
with open(archivo1,'w') as f:   # Creo archivo desde cero, por eso uso "w".
    f.write('Matriz Global\n ===============')

In [10]:
# Defino la matriz elemental "Ke1" de los elementos "0" y "1".
Ke1 = (E*I/L**3)*np.array([[12,6*L,-12,6*L],
                           [6*L,4*(L**2),-6*L,2*(L**2)],
                           [-12,-6*L,12,-6*L],
                           [6*L,2*(L**2),-6*L,4*(L**2)]])

In [11]:
# Defino matriz global "Kg".
Kg = np.zeros([glxn*Nn, glxn*Nn])

# Ensamblo las matrices elementales para obtener la matriz global "Kg".
for e in range(Ne):
    Ke = Ke1
    fe = np.abs(Ke.max()) # Factor de escala, para que los números en "Ke" se lean mejor.
    with open(archivo,'a') as f:   # Voy reescribiendo el archivo con nuevas "Ke", por eso uso "a".
        f.write(f'\nelemento {e}, fe = {fe:4e}\n')
        f.write(f'{Ke/fe}\n')

    for i in range(Nnxe):
        rangoi = np.linspace(i*glxn, (i+1)*glxn-1, Nnxe).astype(int)
        rangoni = np.linspace(MC[e, i]*glxn, (MC[e, i]+1)*glxn-1, Nnxe).astype(int)
        for j in range(Nnxe):
            rangoj = np.linspace(j*glxn, (j+1)*glxn-1, Nnxe).astype(int)
            rangonj = np.linspace(MC[e, j]*glxn, (MC[e, j]+1)*glxn-1, Nnxe).astype(int)
            Kg[np.ix_(rangoni, rangonj)] += Ke[np.ix_(rangoi, rangoj)]
fe = np.abs(Kg.max())
with open(archivo1,'a') as f:   # Reescribo el archivo con la matriz global "Kg" obtenida, por eso uso "a".
    f.write(f'\nMatriz Global, fe = {fe:4e}\n')
    f.write(f'{Kg/fe}\n')

In [12]:
# Llamo al paquete "mef", que contiene la función "solve" que calcula los vectores de fuerzas y torques "F", y de 
# desplazamientos y rotaciones "U", empleando MEF. 
F, U = mef.solve(Kg, r, Fr, s, Us)

In [13]:
print('Las FUERZAS (en lb) y TORQUES (en lb.ft) son:')
print(F)
print('Los DESPLAZAMIENTOS (en ft) y ROTACIONES son:')
print(U)

Las FUERZAS (en lb) y TORQUES (en lb.ft) son:
[[ 1078.125]
 [ 8906.25 ]
 [-1500.   ]
 [    0.   ]
 [  421.875]
 [  937.5  ]]
Los DESPLAZAMIENTOS (en ft) y ROTACIONES son:
[[ 0.        ]
 [ 0.        ]
 [-0.0098195 ]
 [-0.0003055 ]
 [ 0.        ]
 [ 0.00122198]]


In [14]:
# FORMA MÁS LINDA DE PRESENTAR LOS DATOS:
# - "%s" significa que te pone número entero.
# - "%.4f" significa que te pone número con 4 cifras decimales.
# - "%2.4f" y "%7.4f" sólo varían en que con "7" te pone los números alineados respecto del "=" y queda más lindo si 
# llega  a haber uno con signo "-".
for nodo in range(Nn):
    print('Nodo %s     d = %8.4f ft     Phi = %7.4f     F = %11.4f lb     M_Phi = %11.4f lb.ft'%(nodo+1, U[2*nodo]*1000, U[2*nodo+1]*1000, F[2*nodo], F[2*nodo+1]))

Nodo 1     d =   0.0000 ft     Phi =  0.0000     F =   1078.1250 lb     M_Phi =   8906.2500 lb.ft
Nodo 2     d =  -9.8195 ft     Phi = -0.3055     F =  -1500.0000 lb     M_Phi =      0.0000 lb.ft
Nodo 3     d =   0.0000 ft     Phi =  1.2220     F =    421.8750 lb     M_Phi =    937.5000 lb.ft


In [15]:
# Calculo las fuerzas de reacción "R" como "R = F - fm".
fm = fm.reshape([-1,1])
R = F - fm

In [16]:
print('Las REACCIONES de FUERZA (en lb) y TORQUE (en lb.ft) son:')
print(R)

Las REACCIONES de FUERZA (en lb) y TORQUE (en lb.ft) son:
[[1828.125]
 [9843.75 ]
 [   0.   ]
 [   0.   ]
 [1171.875]
 [   0.   ]]
